In [3]:
import asyncio
import json
import datetime
import random
import logging
import paho.mqtt.client as mqtt
from typing import Dict

class MachineSimulator:
    def __init__(self, factory: str, line: str, machine_id: str, mqtt_broker: str = "18.185.31.17"):
        self.factory = factory
        self.line = line
        self.machine_id = machine_id
        self.mqtt_client = mqtt.Client()
        self.mqtt_broker = mqtt_broker
        self.logger = logging.getLogger(f"Machine_{self.machine_id}")

    async def start(self):
        # Connect to the MQTT broker
        self.mqtt_client.connect(self.mqtt_broker, 1883)
        self.mqtt_client.loop_start()  # Start the MQTT loop to send and receive messages
        
        while True:
            try:
                data = self._generate_data()
                self._publish_data(data)  # Publish data to the MQTT broker
                await asyncio.sleep(10)  # Publish every 10 seconds
            except Exception as e:
                self.logger.error(f"Error in machine {self.machine_id}: {e}")

    def _generate_data(self) -> Dict:
        # Use timezone-aware UTC datetime
        return {
            "machineId": self.machine_id,
            "timestamp": datetime.datetime.now(datetime.timezone.utc).isoformat(),
            "measurements": {
                "temperature": round(random.uniform(20, 80), 2),
                "pressure": round(random.uniform(0, 100), 2)
            },
            "status": "RUNNING"
        }

    def _publish_data(self, data: Dict):
        # Publish telemetry data to the MQTT topic
        topic = f"machines/{self.factory}/{self.line}/{self.machine_id}/telemetry"
        self.mqtt_client.publish(topic, json.dumps(data))  # Publish the data as JSON
        self.logger.debug(f"Published data to {topic}: {data}")

# Running the simulator with multiple factories, lines, and machines
async def main():
    logging.basicConfig(level=logging.INFO)
    
    # Create a list of machines under different factories and lines
    simulators = []
    for factory in range(2):  # 2 factories
        for line in range(2):  # 2 production lines per factory
            for machine in range(3):  # 2 machines per line
                simulators.append(MachineSimulator(
                    factory=f"factory{factory}",
                    line=f"line{line}",
                    machine_id=f"machine_{machine}"
                ))
    
    # Start simulators concurrently
    await asyncio.gather(
        *[sim.start() for sim in simulators]
    )

# In VSCode or Jupyter, just use await directly to avoid async loop errors
if __name__ == "__main__":
    await main()  # Directly call await main() instead of using asyncio.run()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it